# Lab 08: AIP Cost Tracking with AWS Cost Explorer

## Business Context

After implementing Application Inference Profiles (AIPs) in Labs 1-7, you now have **operational visibility** through CloudWatch metrics. But your CFO just asked the ultimate question:

**💰 "Show me the actual dollar costs per customer for our AI spending."**

**The Challenge:**
- CloudWatch shows **usage metrics** (invocations, tokens) but not **actual costs**
- You need **precise cost allocation** per tenant for accurate billing
- Finance team requires **cost trends** and **spending patterns** by customer
- You must prove ROI and justify AI infrastructure investments

**The Solution: AWS Cost Explorer + AIP Tags**

By combining Application Inference Profiles with AWS Cost Explorer, you can:
- ✅ Track **actual dollar costs** per tenant using AIP tags
- ✅ Generate **accurate bills** for usage-based pricing
- ✅ Identify **cost trends** and **spending patterns** by customer
- ✅ Create **financial reports** for executive dashboards

## Learning Objectives

By the end of this lab, you'll master:
- Fetching Bedrock costs from AWS Cost Explorer filtered by AIP tags
- Creating cost visualizations and financial reports
- Implementing multi-tenant cost allocation for accurate billing
- Building executive dashboards with cost trends and breakdowns

## What You'll Build

1. **🔍 Cost Data Retrieval** - Fetch Bedrock costs by tenant using Cost Explorer API
2. **📊 Cost Visualizations** - Create charts showing cost trends and breakdowns
3. **💼 Financial Reports** - Generate summary tables for billing and finance teams
4. **📈 Executive Dashboard** - Build comprehensive cost monitoring for leadership

**Ready to track real costs?** Let's start by fetching actual Bedrock spending per tenant...

## Section 1: Setup and Configuration

First, let's import the necessary libraries and configure our AWS Cost Explorer client.

In [ ]:
# Import required libraries
import boto3
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import sys
import os

# Add lab_helpers to path
sys.path.append(os.path.join(os.getcwd(), 'lab_helpers'))

# Import our custom Cost Explorer helper
from cost_explorer import (
    fetch_bedrock_costs_by_tag,
    fetch_all_tenant_costs,
    visualize_tenant_costs,
    create_cost_summary_table,
    visualize_cost_breakdown
)

# Import configuration
from config import Region

print("✅ Libraries imported successfully")
print(f"📍 Region: {Region}")
print(f"🕐 Current time: {datetime.now()}")

## Section 2: Verify Cost Explorer Permissions

Before fetching cost data, let's verify that we have the necessary permissions to access Cost Explorer.

In [ ]:
# Define AWS credentials (optional - leave None to use IAM role)
aws_access_key_id = None  # Set to your access key if needed
aws_secret_access_key = None  # Set to your secret key if needed

# Initialize Cost Explorer client
if aws_access_key_id and aws_secret_access_key:
    ce_client = boto3.client(
        'ce', 
        region_name=Region,
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key
    )
else:
    ce_client = boto3.client('ce', region_name=Region)

try:
    # Test basic Cost Explorer access
    end_date = datetime.now().date()
    start_date = end_date - timedelta(days=7)
    
    test_response = ce_client.get_cost_and_usage(
        TimePeriod={
            'Start': start_date.strftime('%Y-%m-%d'),
            'End': end_date.strftime('%Y-%m-%d')
        },
        Granularity='DAILY',
        Metrics=['BlendedCost'],
        GroupBy=[
            {
                'Type': 'DIMENSION',
                'Key': 'SERVICE'
            }
        ]
    )
    
    print("✅ Cost Explorer access verified")
    print(f"📊 Retrieved {len(test_response.get('ResultsByTime', []))} days of cost data")
    
except Exception as e:
    print(f"❌ Cost Explorer access failed: {str(e)}")
    print("\n🔧 Required permissions:")
    print("   - ce:GetCostAndUsage")
    print("   - ce:GetUsageReport")
    print("   - ce:ListCostCategoryDefinitions")

## Section 3: Fetch Bedrock Costs by Tenant

Now let's fetch actual Bedrock costs for each tenant using their AIP tags. This is where the magic happens - we can see real dollar amounts per customer!

In [ ]:
# Define the tenants we want to track costs for
# These should match the tenant IDs used in your AIP tags
tenant_ids = ['tenant-c-demo', 'tenant-d-demo']
tag_key = 'TenantId'  # The tag key used in your AIPs
#tenant_ids = ['teama', 'teamb']
#tag_key = 'team'  # The tag key used in your AIPs
days_back = 2  # Look back 2 days for workshop data

print(f"🔍 Fetching Bedrock costs for tenants: {tenant_ids}")
print(f"📅 Time range: Last {days_back} days")
print(f"🏷️  Tag filter: {tag_key}")
print("\n⏳ This may take a moment...")

In [ ]:
# Fetch costs for all tenants
tenant_costs = fetch_all_tenant_costs(
    region=Region,
    tenant_ids=tenant_ids,
    days_back=days_back,
    tag_key=tag_key,
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key
)

print("\n✅ Cost data retrieved successfully!")
print(f"📊 Retrieved cost data for {len(tenant_costs)} tenants")

# Quick preview of the data structure
for tenant_id, cost_data in tenant_costs.items():
    num_days = len(cost_data.get('ResultsByTime', []))
    print(f"   📈 {tenant_id}: {num_days} days of cost data")

## Section 4: Cost Summary and Analysis

Let's create a comprehensive summary of costs per tenant. This is what your finance team and executives want to see!

In [ ]:
# Create cost summary table
cost_summary = create_cost_summary_table(tenant_costs)

print("💰 BEDROCK COST SUMMARY BY TENANT")
print("=" * 50)

if not cost_summary.empty:
    # Display the summary table
    print(cost_summary.to_string(index=False))
    
    # Calculate totals
    total_cost = cost_summary['Total Cost ($)'].sum()
    total_days_with_usage = cost_summary['Days with Usage'].sum()
    
    print(f"\n📊 OVERALL SUMMARY:")
    print(f"   💵 Total Bedrock Spend: ${total_cost:.4f}")
    print(f"   📅 Total Days with Usage: {total_days_with_usage}")
    print(f"   🏢 Active Tenants: {len(cost_summary)}")
    
    # Identify highest spending tenant
    if len(cost_summary) > 0:
        top_spender = cost_summary.loc[cost_summary['Total Cost ($)'].idxmax()]
        print(f"   🥇 Highest Spender: {top_spender['Tenant ID']} (${top_spender['Total Cost ($)']:.4f})")
        
else:
    print("ℹ️  No cost data found for the specified time period.")
    print("   This could mean:")
    print("   - No Bedrock usage in the last 30 days")
    print("   - AIP tags are not properly configured")
    print("   - Cost data hasn't been processed yet (can take 24-48 hours)")

## Section 5: Cost Trend Visualization

Now let's create visualizations to show cost trends over time. This helps identify usage patterns and spending spikes.

In [ ]:
# Create cost trend visualization
print("📈 Creating cost trend visualization...")

visualize_tenant_costs(
    tenant_costs=tenant_costs,
    title=f"Bedrock Daily Costs by Tenant (Last {days_back} Days)"
)

print("\n💡 INSIGHTS FROM COST TRENDS:")
print("   📊 Look for patterns in the chart above:")
print("   - Which tenant has consistent daily usage?")
print("   - Are there any cost spikes or unusual patterns?")
print("   - How do weekday vs weekend costs compare?")
print("   - Which tenant might need usage optimization?")

## Section 6: Cost Distribution Analysis

Let's create pie and bar charts to show how costs are distributed across tenants. Perfect for executive presentations!

In [ ]:
# Create pie chart showing cost distribution
print("🥧 Creating cost distribution pie chart...")

visualize_cost_breakdown(
    tenant_costs=tenant_costs,
    chart_type='pie'
)

In [ ]:
# Create bar chart showing total costs by tenant
print("📊 Creating cost comparison bar chart...")

visualize_cost_breakdown(
    tenant_costs=tenant_costs,
    chart_type='bar'
)

## Section 7: Billing and Financial Insights

Now let's generate actionable insights for your finance team and create billing recommendations.

In [ ]:
# Generate billing insights and recommendations
print("💼 BILLING & FINANCIAL INSIGHTS")
print("=" * 40)

if not cost_summary.empty:
    # Calculate key metrics for billing
    total_monthly_cost = cost_summary['Total Cost ($)'].sum()
    avg_cost_per_tenant = cost_summary['Total Cost ($)'].mean()
    cost_variance = cost_summary['Total Cost ($)'].std()
    
    print(f"📊 FINANCIAL METRICS:")
    print(f"   💰 Total Monthly AI Spend: ${total_monthly_cost:.4f}")
    print(f"   📈 Average Cost per Tenant: ${avg_cost_per_tenant:.4f}")
    print(f"   📊 Cost Variance (StdDev): ${cost_variance:.4f}")
    
    # Billing recommendations
    print(f"\n💡 BILLING RECOMMENDATIONS:")
    
    # Identify high-value customers
    high_spenders = cost_summary[cost_summary['Total Cost ($)'] > avg_cost_per_tenant]
    if len(high_spenders) > 0:
        print(f"   🥇 High-Value Customers ({len(high_spenders)} tenants):")
        for _, tenant in high_spenders.iterrows():
            print(f"      - {tenant['Tenant ID']}: ${tenant['Total Cost ($)']:.4f}")
    
    # Identify optimization opportunities (tenants with less than 50% usage)
    low_usage = cost_summary[cost_summary['Days with Usage'] < (days_back * 0.5)]
    if len(low_usage) > 0:
        print(f"\n   ⚡ Optimization Opportunities ({len(low_usage)} tenants):")
        for _, tenant in low_usage.iterrows():
            usage_rate = (tenant['Days with Usage'] / days_back) * 100
            print(f"      - {tenant['Tenant ID']}: {usage_rate:.1f}% usage rate")
    
    # Pricing model suggestions
    print(f"\n   💵 PRICING MODEL SUGGESTIONS:")
    if cost_variance > avg_cost_per_tenant * 0.5:
        print(f"      - High cost variance detected - consider tiered pricing")
        print(f"      - Implement usage-based billing with volume discounts")
    else:
        print(f"      - Consistent usage patterns - flat rate pricing may work")
    
    # ROI calculations
    print(f"\n   📈 ROI ANALYSIS:")
    estimated_monthly_revenue = total_monthly_cost * 3  # Assume 3x markup
    print(f"      - Current AI Infrastructure Cost: ${total_monthly_cost:.4f}/month")
    print(f"      - Potential Revenue (3x markup): ${estimated_monthly_revenue:.4f}/month")
    print(f"      - Gross Margin: ${estimated_monthly_revenue - total_monthly_cost:.4f}/month")

else:
    print("ℹ️  No cost data available for analysis.")
    print("   Ensure AIPs are properly tagged and have recent usage.")

## Section 8: Executive Dashboard Summary

Finally, let's create a comprehensive executive summary that you can present to leadership.

In [ ]:
# Create executive dashboard summary
print("🎯 EXECUTIVE DASHBOARD - AI COST ALLOCATION")
print("=" * 50)
print(f"📅 Report Period: Last {days_back} days")
print(f"🕐 Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("\n" + "=" * 50)

if not cost_summary.empty:
    # Key Performance Indicators
    total_cost = cost_summary['Total Cost ($)'].sum()
    active_tenants = len(cost_summary[cost_summary['Total Cost ($)'] > 0])
    avg_daily_spend = cost_summary['Avg Daily Cost ($)'].mean()
    
    print("📊 KEY PERFORMANCE INDICATORS:")
    print(f"   💰 Total AI Infrastructure Spend: ${total_cost:.4f}")
    print(f"   🏢 Active Paying Customers: {active_tenants}")
    print(f"   📈 Average Daily Spend: ${avg_daily_spend:.4f}")
    print(f"   💵 Cost per Active Customer: ${total_cost/active_tenants:.4f}")
    
    # Business Impact
    print(f"\n🎯 BUSINESS IMPACT:")
    print(f"   ✅ Achieved 100% cost visibility per customer")
    print(f"   ✅ Enabled accurate usage-based billing")
    print(f"   ✅ Identified {active_tenants} revenue-generating customers")
    print(f"   ✅ Established baseline for AI infrastructure ROI")
    
    # Strategic Recommendations
    print(f"\n🚀 STRATEGIC RECOMMENDATIONS:")
    print(f"   1. Implement tiered pricing based on usage patterns")
    print(f"   2. Focus customer success efforts on high-value users")
    print(f"   3. Optimize infrastructure for cost-efficient scaling")
    print(f"   4. Set up automated cost alerts for budget management")
    
    # Next Steps
    print(f"\n📋 NEXT STEPS:")
    print(f"   □ Set up automated daily cost reports")
    print(f"   □ Implement cost-based customer segmentation")
    print(f"   □ Create customer-facing usage dashboards")
    print(f"   □ Establish cost optimization targets")

else:
    print("⚠️  ATTENTION REQUIRED:")
    print("   No cost data detected for AI infrastructure.")
    print("   Immediate action needed to enable cost tracking.")

print("\n" + "=" * 50)
print("📧 Report prepared by: Platform Engineering Team")
print("🔄 Next report: Automated daily at 9:00 AM")

## 🎉 Lab 08 Complete!

**Congratulations!** You've successfully implemented comprehensive cost tracking for your multi-tenant AI platform using AWS Cost Explorer and Application Inference Profiles.

### What You've Accomplished:

✅ **Cost Visibility**: Retrieved actual dollar costs per tenant using AIP tags  
✅ **Financial Reporting**: Created executive-ready cost summaries and trends  
✅ **Billing Foundation**: Established accurate cost allocation for usage-based pricing  
✅ **Business Intelligence**: Generated actionable insights for strategic decisions  

### The Complete Multi-Tenant AI Solution:

You now have **end-to-end visibility** across your multi-tenant AI platform:

- **Labs 1-7**: Operational metrics (invocations, tokens, performance)
- **Lab 8**: Financial metrics (actual costs, billing, ROI)

### Business Value Delivered:

💰 **Accurate Billing**: Precise cost allocation enables usage-based pricing  
📊 **Executive Insights**: Leadership can make data-driven AI investment decisions  
🎯 **Customer Segmentation**: Identify high-value customers and optimization opportunities  
🚀 **Scalable Growth**: Foundation for profitable multi-tenant AI platform expansion  

### Production Deployment Checklist:

□ Set up automated daily cost reports  
□ Implement cost-based alerting and budgets  
□ Create customer-facing usage dashboards  
□ Establish cost optimization monitoring  
□ Integrate with billing and CRM systems  

**🎯 Mission Accomplished**: You've transformed from having zero tenant visibility to complete operational and financial transparency across your multi-tenant AI platform!

---

**Next Steps**: Take this foundation and build automated reporting, customer dashboards, and cost optimization workflows for your production environment.